# [Project 3: Camera Calibration and Fundamental Matrix Estimation with RANSAC](https://www.cc.gatech.edu/~hays/compvision/proj3)


(1) Camera Projection Matrix  
(2) Fundamental Matrix Estimation  
(3) Fundamental Matrix with RANSAC   
(4) Comparison of the results from (2) and (3)   
(5) Using F-Matrix Estimation w/ RANSAC for Visual Odometry   

## Setup

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import cv2
import numpy as np
import matplotlib.pyplot as plt
from vision.utils import (
    verify,
    evaluate_points,
    visualize_points,
    visualize_points_image,
    plot3dview,
    load_image,
    draw_epipolar_lines,
    get_matches,
    show_correspondence2,
)

## Part 1: Camera Projection Matrix

In [ ]:
# Load the data
img_path = "data/CCB_GaTech/pic_a.jpg"
points_2d = np.loadtxt("data/CCB_GaTech/pts2d-norm-pic_a.txt")
points_3d = np.loadtxt("data/CCB_GaTech/pts3d-norm.txt")

# (Optional) Uncomment these four lines once you have your code working with the easier, normalized points above.
# points_2d = np.loadtxt('../data/CCB_GaTech/pts2d-pic_b.txt')
# points_3d = np.loadtxt('../data/CCB_GaTech/pts3d.txt')

### Calculate the projection matrix given corresponding 2D & 3D points

In [ ]:
from vision.part1_projection_matrix import (
    calculate_projection_matrix,
    calculate_camera_center,
)

from tests.test_part1 import (
    test_projection,
    test_calculate_projection_matrix,
    test_calculate_camera_center,
)

In [ ]:
print("projection():", verify(test_projection))

In [ ]:
M = calculate_projection_matrix(points_2d, points_3d)
print("The projection matrix is\n", M)

[projected_2d_pts, residual] = evaluate_points(M, points_2d, points_3d)
print("The total residual is {:f}".format(residual))
plt.figure()
plt.imshow(load_image(img_path))
visualize_points(points_2d, projected_2d_pts)

In [ ]:
print("calculate_projection_matrix():", verify(test_calculate_projection_matrix))

### Calculate the camera center using M found from the previous step

In [ ]:
center = calculate_camera_center(M)
print(
    "The estimated location of the camera is <{:.4f}, {:.4f}, {:.4f}>".format(*center)
)
plt.figure()
plt.imshow(load_image(img_path))
ax = plot3dview(points_3d, center)

In [ ]:
print("test_calculate_camera_center():", verify(test_calculate_camera_center))

## Camera Calibration for Argoverse image data
We'll now estimate the position of a camera mounted on an autonomous vehicle, using data from Argoverse. We'll use images from the "ring front center" camera, which faces forward.


<img src="https://www.argoverse.org/assets/images/reference_images/O2V4_vehicle_annotation.jpg" alt="Drawing" style="width: 400px;"/>


In [ ]:
# Argoverse Scene 3
img_path = "data/argoverse_log_d60558d2_pair3/pic3.jpg"
points_2d = np.loadtxt("data/argoverse_log_d60558d2_pair3/points_2d.txt")
points_3d = np.loadtxt("data/argoverse_log_d60558d2_pair3/points_3d.txt")
# # # Argoverse Scene 2
# img_path = '../data/argoverse_log_d60558d2_pair2/pic2.jpg'
# points_2d = np.loadtxt('../data/argoverse_log_d60558d2_pair2/points_2d.txt')
# points_3d = np.loadtxt('../data/argoverse_log_d60558d2_pair2/points_3d.txt')

In [ ]:
M = calculate_projection_matrix(points_2d, points_3d)
print("The projection matrix is\n", M)

[projected_2d_pts, residual] = evaluate_points(M, points_2d, points_3d)
print("The total residual is {:f}".format(residual))
plt.figure()
plt.imshow(load_image(img_path))
visualize_points(points_2d, projected_2d_pts)

For these 2d-3d pairs, the "world" frame is defined as the "ego-vehicle" frame, where the origin is at the center of the back axle of the vehicle.

Thus, if your camera center estimate is correct, it should tell you how far to move forward (+x) and how far to move left (+y) and move up (+z) to reach teh camera's position.


The "egovehicle" coordinate system and "camera" coordinate system:
<img width="300"  src="https://user-images.githubusercontent.com/16724970/108759169-034e6180-751a-11eb-8a06-fbe344f1ee68.png">
<img width="300" src="https://user-images.githubusercontent.com/16724970/108759182-06495200-751a-11eb-8162-8b17f9cdee4b.png">

In [ ]:
center = calculate_camera_center(M)
print(
    "The estimated location of the camera is <{:.4f}, {:.4f}, {:.4f}>".format(*center)
)
plt.figure()
plt.imshow(load_image(img_path))
ax = plot3dview(points_3d, center)
ax.view_init(elev=15, azim=180)

## Part 2: Fundamental Matrix Estimation
We'll now solve for the Fundamental Matrix by implementing [Hartley's 8-Point algorithm](https://www.cse.unr.edu/~bebis/CS485/Handouts/hartley.pdf).

In [ ]:
from vision.part2_fundamental_matrix import estimate_fundamental_matrix

In [ ]:
# Load the data
points_2d_pic_a = np.loadtxt("data/CCB_GaTech/pts2d-pic_a.txt")
points_2d_pic_b = np.loadtxt("data/CCB_GaTech/pts2d-pic_b.txt")
img_a = load_image("data/CCB_GaTech/pic_a.jpg")
img_b = load_image("data/CCB_GaTech/pic_b.jpg")

### Estimate fundamental matrix

In [ ]:
from tests.test_part2 import (
    test_estimate_fundamental_matrix,
    test_normalize_points,
    test_unnormalize_F,
)

print("test_estimate_fundamental_matrix():", verify(test_estimate_fundamental_matrix))
print("test_normalize_points():", verify(test_normalize_points))
print("test_unnormalize_F():", verify(test_unnormalize_F))

In [ ]:
F = estimate_fundamental_matrix(points_2d_pic_a, points_2d_pic_b)

# Draw epipolar lines using the fundamental matrix
draw_epipolar_lines(F, img_a, img_b, points_2d_pic_a, points_2d_pic_b, figsize=(13, 4))

## Part 3: Fundamental Matrix with RANSAC (Szeliski 6.1.4)

**Mount Rushmore**: This pair is easy, and most of the initial matches are correct. The base fundamental matrix estimation without coordinate normalization will work fine with RANSAC. 

**Notre Dame**: This pair is difficult because the keypoints are largely on the same plane. Still, even an inaccurate fundamental matrix can do a pretty good job of filtering spurious matches.  

**Gaudi**: This pair is difficult and doesn't find many correct matches unless you run at high resolution, but that will lead to tens of thousands of SIFT features, which will be somewhat slow to process. Normalizing the coordinates seems to make this pair work much better.  

**Woodruff**: This pair has a clearer relationship between the cameras (they are converging and have a wide baseline between them). The estimated fundamental matrix is less ambiguous and you should get epipolar lines qualitatively similar to part 2 of the project.

In [ ]:
from vision.part3_ransac import (
    calculate_num_ransac_iterations,
    ransac_fundamental_matrix,
)

In [ ]:
# Load the data
# Mount Rushmore
# pic_a = load_image('../data/Mount_Rushmore/9193029855_2c85a50e91_o.jpg'); scale_a = 0.25
# pic_b = load_image('../data/Mount_Rushmore/7433804322_06c5620f13_o.jpg'); scale_b = 0.37
# n_feat = 5e4

# Notre Dame
pic_a = load_image("data/Notre_Dame/921919841_a30df938f2_o.jpg")
scale_a = 0.5
pic_b = load_image("data/Notre_Dame/4191453057_c86028ce1f_o.jpg")
scale_b = 0.5
n_feat = 4e3

# Gaudi
# pic_a = load_image('../data/Episcopal_Gaudi/3743214471_1b5bbfda98_o.jpg'); scale_a = 0.8
# pic_b = load_image('../data/Episcopal_Gaudi/4386465943_8cf9776378_o.jpg'); scale_b = 1.0
# n_feat = 2e4

# Woodruff
# pic_a = load_image('../data/Woodruff_Dorm/wood1.jpg'); scale_a = 0.65
# pic_b = load_image('../data/Woodruff_Dorm/wood2.jpg'); scale_b = 0.65
# n_feat = 5e4

pic_a = cv2.resize(pic_a, None, fx=scale_a, fy=scale_a)
pic_b = cv2.resize(pic_b, None, fx=scale_b, fy=scale_b)

In [ ]:
# Finds matching points in the two images using OpenCV's implementation of SIFT.
# There can still be many spurious matches, though.
points_2d_pic_a, points_2d_pic_b = get_matches(pic_a, pic_b, n_feat)
print("Found {:d} possibly matching features".format(len(points_2d_pic_a)))
match_image = show_correspondence2(
    pic_a,
    pic_b,
    points_2d_pic_a[:, 0],
    points_2d_pic_a[:, 1],
    points_2d_pic_b[:, 0],
    points_2d_pic_b[:, 1],
)
plt.figure()
plt.imshow(match_image)
plt.tight_layout()

### Calculate the Fundamental Matrix using RANSAC
Compare your results on the Notre Dame image pair below to your results from Project 2. How accurate do the point correspondences look now? 

In [ ]:
F, matched_points_a, matched_points_b = ransac_fundamental_matrix(
    points_2d_pic_a, points_2d_pic_b
)

# Draw the epipolar lines on the images and corresponding matches
match_image = show_correspondence2(
    pic_a,
    pic_b,
    matched_points_a[:, 0],
    matched_points_a[:, 1],
    matched_points_b[:, 0],
    matched_points_b[:, 1],
)
plt.figure()
plt.imshow(match_image)
plt.tight_layout()

In [ ]:
draw_epipolar_lines(
    F, pic_a, pic_b, matched_points_a, matched_points_b, figsize=(12, 8)
)

In [ ]:
from tests.test_part3 import (
    test_calculate_num_ransac_iterations,
    test_ransac_fundamental_matrix,
)

print(
    "test_calculate_num_ransac_iterations():",
    verify(test_calculate_num_ransac_iterations),
)
print("test_ransac_fundamental_matrix():", verify(test_ransac_fundamental_matrix))

## Part 4: Performance Comparison
We'll now test the quality of Fundamental matrices we can compute with and without RANSAC on an image pair from the [Argoverse](https://www.argoverse.org/) autonomous driving dataset. Does RANSAC improve the results?

In [ ]:
from vision.part3_ransac import (
    calculate_num_ransac_iterations,
    ransac_fundamental_matrix,
)

In [ ]:
pic_a = load_image(
    "data/argoverse_log_273c1883/ring_front_center_315975640448534784.jpg"
)
scale_a = 0.5
pic_b = load_image(
    "data/argoverse_log_273c1883/ring_front_center_315975643412234000.jpg"
)
scale_b = 0.5

n_feat = 4e3
num_matches_to_plot = 50

pic_a = cv2.resize(pic_a, None, fx=scale_a, fy=scale_a)
pic_b = cv2.resize(pic_b, None, fx=scale_b, fy=scale_b)

In [ ]:
points_2d_pic_a, points_2d_pic_b = get_matches(pic_a, pic_b, n_feat)

print("Found {:d} possibly matching features".format(len(points_2d_pic_a)))
match_image = show_correspondence2(
    pic_a,
    pic_b,
    points_2d_pic_a[:num_matches_to_plot, 0],
    points_2d_pic_a[:num_matches_to_plot, 1],
    points_2d_pic_b[:num_matches_to_plot, 0],
    points_2d_pic_b[:num_matches_to_plot, 1],
)
plt.figure(figsize=(14, 6))
plt.imshow(match_image)

### Without RANSAC Estimation
If we ignore RANSAC and use only our implementation from Part2, we get the following results:

In [ ]:
F_wo_ransac = estimate_fundamental_matrix(points_2d_pic_a, points_2d_pic_b)

# Draw epipolar lines using the fundamental matrix
draw_epipolar_lines(
    F_wo_ransac, pic_a, pic_b, points_2d_pic_a, points_2d_pic_b, figsize=(13, 4)
)

### With Ransac Estimation
Now we'll use our RANSAC implementation from Part 3. Where does the epipole fall in the left image? (think about what it represents). The camera is mounted on an autonomous vehicle identical to the vehicle seen up ahead in the left image.

In [ ]:
F, matched_points_a, matched_points_b = ransac_fundamental_matrix(
    points_2d_pic_a, points_2d_pic_b
)

draw_epipolar_lines(
    F, pic_a, pic_b, matched_points_a, matched_points_b, figsize=(13, 4)
)

match_image = show_correspondence2(
    pic_a,
    pic_b,
    points_2d_pic_a[:num_matches_to_plot, 0],
    points_2d_pic_a[:num_matches_to_plot, 1],
    points_2d_pic_b[:num_matches_to_plot, 0],
    points_2d_pic_b[:num_matches_to_plot, 1],
)
plt.figure(figsize=(14, 4))
plt.imshow(match_image)

## Part 5: Visual Odometry

In [ ]:
from vision.vo import get_visual_odometry, plot_poses

For the following gif, we try to obtain the visual odometry of a camera mounted on a robot from the individual image frames.

![VO](https://user-images.githubusercontent.com/16724970/100487935-34fd8b00-30d9-11eb-9941-7735fcae445c.gif "VO")

In [ ]:
# This may take a few minutes to run across 20 image frames from the Argoverse dataset
poses_wTi = get_visual_odometry()

In [ ]:
plot_poses(poses_wTi)